In [1]:

import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from model_arch import *
import csv, sys, pdb
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix


In [ ]:
file_rigid_accel2 = "/home/togzhan/sample/data/rigid_accel2.csv"
file_rigid_accel = "/home/togzhan/sample/data/rigid_accel.csv"
file_rigid_pressure = "/home/togzhan/sample/data/rigid_pressure.csv"
file_rigid_imu = "/home/togzhan/sample/data/rigid_imu.csv"


file_soft_accel2 = "/home/togzhan/sample/data/soft_accel2.csv"
file_soft_accel = "/home/togzhan/sample/data/soft_accel.csv"
file_soft_pressure = "/home/togzhan/sample/data/soft_pressure.csv"
file_soft_imu = "/home/togzhan/sample/data/soft_imu.csv"

df_accel2_rigid = pd.read_csv(file_rigid_accel2, sep = ',', header = None)
df_accel_rigid = pd.read_csv(file_rigid_accel, sep = ',', header = None)
df_pressure_rigid = pd.read_csv(file_rigid_pressure, sep = ',', header = None)
df_imu_rigid = pd.read_csv(file_rigid_imu, sep = ',', header = None)

df_accel2_soft = pd.read_csv(file_soft_accel2, sep = ',', header = None)
df_accel_soft = pd.read_csv(file_soft_accel, sep = ',', header = None)
df_pressure_soft = pd.read_csv(file_soft_pressure, sep = ',', header = None)
df_imu_soft = pd.read_csv(file_soft_imu, sep = ',', header = None)



In [2]:
dftrain = "/home/togzhan/sample/data/data/df_train4.csv"
dfeval = "/home/togzhan/sample/data/data/df_eval4.csv"

df_train = pd.read_csv(dftrain, sep = ',', header = None)
df_eval = pd.read_csv(dfeval, sep = ',', header = None)

In [ ]:
#######################################################
def normalize(data):
    m = data.max()
    data = pd.DataFrame(data)
#     df_max = [420, 420, 420, 418, 418, 450, 432, 432, 432, 390, 472, 480, 480, 480, 435, 433, 501, 405, 400, 400, 400 ,400, 370, 455 ,444, 310, 445]
    df_max = [450]
    df_max = pd.DataFrame(df_max)
    df_min = 0;
    p = pd.DataFrame()
    k = []
    for i in range(27):
        k = np.divide(data.values[:,i].astype('float'), df_max.values[0].astype('float'))
        k = pd.DataFrame(k)    
        p = pd.concat([p, k], axis = 1)
        
    data = p
    data = pd.DataFrame(p)
    return data
#######################################################


In [ ]:
def combine_the_data(df_accel2, df_pressure, df_imu, inp):
    df_pressure_n = normalize(df_pressure)
    dP = pd.DataFrame()
    dA = pd.DataFrame()
    dI = pd.DataFrame()
    D = pd.DataFrame()
    data = pd.DataFrame()
    df_accelT = df_accel2.transpose()
    df_pressure_nT = df_pressure_n.transpose()
    df_imuT = df_imu.transpose()
    for i in range(50):
        P = pd.DataFrame()
        A = pd.DataFrame()
        I = pd.DataFrame()
        for j in range(27):
            p = []
            p = df_pressure_nT.values[j, i*744:(i+1)*744]
            P = pd.concat([P, pd.DataFrame(p)], axis = 0, ignore_index=True, sort = False )
        for j in range(200):
            a = []
            a = df_accelT.values[j, i*744:(i+1)*744]
            A = pd.concat([A, pd.DataFrame(a)], axis = 0, ignore_index=True, sort = False)
        for j in range(18):
            ii = []
            ii = df_imuT.values[j, i*744:(i+1)*744]
            I = pd.concat([I, pd.DataFrame(ii)], axis = 0, ignore_index=True, sort = False)
        
        D = pd.concat([P, A, I], axis = 0)
#         print I
#         print D
#         print A
        data = pd.concat([data, D], axis = 1)
#         dP = pd.concat([dP, P], axis = 1)
#         dA = pd.concat([dA, A], axis = 1)
#         dI = pd.concat([dI, I], axis = 1)
    return data

In [ ]:
df_bilabel = []
for i in range(100):
    if(i<50):
        df_bilabel.append(0)
    else:
        df_bilabel.append(1)
        
df_bilabel = pd.DataFrame(df_bilabel, index=None)

k = 0
labels = []
for i in range(100):
    if(np.mod(i,5) == 0):
        k = k +1
    labels.append(k)        
df_label = pd.DataFrame(labels, index=None)

# labels - 1st number 0 or 1 0 - rigid 1 - soft, second number defines the object
df_newlabel = df_bilabel.astype(str) + "_" + df_label.astype(str)


In [ ]:
print df_bilabel.shape
print df_data.shape
a = pd.DataFrame()
a = pd.concat([df_bilabel, df_label], axis = 1, sort=False)
df_data =  df_data.reset_index(drop=True)
df_data

In [ ]:
df_train.to_csv(path_or_buf="/home/togzhan/sample/data/data/df_train4.csv", index=False)
df_eval.to_csv(path_or_buf="/home/togzhan/sample/data/data/df_eval4.csv", index=False)

In [ ]:
data_rigid = pd.DataFrame()
data_soft = pd.DataFrame()

data_rigid = combine_the_data(df_accel2_rigid, df_pressure_rigid, df_imu_rigid, 1)

data_soft = combine_the_data(df_accel2_soft, df_pressure_soft, df_imu_soft, 1)

data_rigid = data_rigid.transpose()
data_soft = data_soft.transpose()


In [ ]:
df_data = pd.concat([data_rigid, data_soft], axis = 0, sort = False)
df_data.reset_index(drop=True, inplace=True) 


In [ ]:
df_data_l = pd.DataFrame()
df_data_l = pd.concat([df_data, df_bilabel], axis = 1, sort = False)


In [ ]:
df_train, df_eval = train_test_split(df_data_l, test_size=1.0/3, random_state=777, stratify=df_newlabel.values)


In [ ]:
a = df_train.shape

label_train = df_train.values[:, a[1]-1]
label_eval = df_eval.values[:, a[1]-1]
df_train_e = pd.DataFrame(df_train.values[:, 1:a[1]-18*744-1])
df_eval_e = pd.DataFrame(df_eval.values[:, 1:a[1]-18*744-1])


In [ ]:
m = "/home/togzhan/sample/data/data/biclass_adaboost1_max_depth150_n_estimators"
model = torch.load(m)-
model.predict(test_x[4].reshape(1,-1))

In [ ]:
###### LOAD previously saved and shuffled data .

import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from model_arch import *
import csv, sys, pdb
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

df_train4 = "/home/togzhan/sample/data/data/df_train4.csv"
df_eval4 = "/home/togzhan/sample/data/data/df_eval4.csv"

df_train = pd.read_csv(df_train4, sep = ',', header = None)
df_eval = pd.read_csv(df_eval4, sep = ',', header = None)


In [4]:
train_x	= torch.FloatTensor(df_train.values[:,:-1].astype('float64'))
test_x = torch.FloatTensor(df_eval.values[:,:-1].astype('float64'))

In [5]:
train_y = torch.FloatTensor(df_train.values[:,-1].astype('float64'))
test_y = torch.FloatTensor(df_eval.values[:,-1].astype('float64'))


In [ ]:
class TwoLayerNet(torch.nn.Module):
  def __init__(self, D_in, H, D_out, drop=0.0):
    """
    In the constructor we instantiate two nn.Linear modules and assign them as member variables.
    """
    super(TwoLayerNet, self).__init__()
    self.linear1 = torch.nn.Linear(D_in, H)
    self.linear1_1 = torch.nn.Linear(H, H)
    self.linear1_2 = torch.nn.Linear(H, H)
    self.linear2 = torch.nn.Linear(H, D_out)
    self.drop1   = torch.nn.Dropout(p=drop)
    self.sigmoid = torch.nn.Sigmoid()

  def forward(self, x):
    """
    In the forward function we accept a Tensor of input data and we must return
    a Tensor of output data. We can use Modules defined in the constructor as
    well as arbitrary operators on Tensors.
    """
    h_relu = self.drop1(self.linear1(x)).clamp(min=0)
    h_relu = self.drop1(self.linear1_1(h_relu)).clamp(min=0)
    h_relu = self.drop1(self.linear1_2(h_relu)).clamp(min=0)
    y_pred = self.sigmoid(self.linear2(h_relu))
    return y_pred


class ConvNet(torch.nn.Module):
  def __init__(self, H_in, W_in, D_out, ch_out, H, kernel_w=3, stride_size=1, drop=0.0):
    """
    In the constructor we instantiate Convlolutional and one Linear modules and assign them as member variables.
    """
    super(ConvNet, self).__init__()
    self.conv = torch.nn.Sequential(
                  torch.nn.Conv2d(1, ch_out, (H_in, kernel_w), stride=stride_size),
                  torch.nn.ReLU(),
                  torch.nn.Conv2d(ch_out, ch_out, (H_in, kernel_w), stride=stride_size),
                  torch.nn.ReLU(),
                  torch.nn.Conv2d(ch_out, ch_out, (H_in, kernel_w), stride=stride_size),
                  torch.nn.ReLU()
                )
    W_out = ((W_in - kernel_w) // stride_size + 1)
    W_out = ((W_out - kernel_w) // stride_size + 1)
    W_out = ch_out * ((W_out - kernel_w) // stride_size + 1)
    #pdb.set_trace()
    #self.linear1 = torch.nn.Linear(W_out, D_out)
    self.linear1 = torch.nn.Linear(W_out, H)
    self.linear2 = torch.nn.Linear(H, D_out)
    self.drop1   = torch.nn.Dropout(p=drop)
    self.sigmoid = torch.nn.Sigmoid()

  def forward(self, x):
    """
    In the forward function we accept a Tensor of input data and we must return
    a Tensor of output data. We can use Modules defined in the constructor as
    well as arbitrary operators on Tensors.
    """
    (b,w) = x.shape
    h_cnn = self.drop1(self.conv(x.view(b,1,1,w)))
    b, c, h, w = h_cnn.size()
    #pdb.set_trace()
    h_relu = self.drop1(self.linear1(h_cnn.view(b,-1))).clamp(min=0)
    y_pred = self.sigmoid(self.linear2(h_relu))
    return y_pred

In [ ]:
#CNN Model
dtype = torch.float
epochs = 5000
N = 16 #batch size
D_in = train_x.shape[1]
D_out = 1
drop = 0.4
#FFN Model
H = 512 #hidden size
device = torch.device("cpu")
print(device)
ch_out = 4
kernel_w = 16
stride = 4
###################################################################################################

W_in = train_x.shape[1]
model = ConvNet(1, W_in, D_out, ch_out, H, kernel_w, stride, drop).to(device)
#model_path = "./model/biclass_cnn_l1_h"+str(H)+"_bs"+str(N)+"_c"+str(ch_out)+"_k"+str(kernel_w)+"_s"+str(stride)+"_drop"+str(drop)+"_epoch"+str(epochs)+"_accX"
model_path = "/home/togzhan/sample/data/data/models/biclass_cnn_l1_h"+str(H)+"_bs"+str(N)+"_c"+str(ch_out)+"_k"+str(kernel_w)+"_s"+str(stride)+"_drop"+str(drop)+"_epoch"+str(epochs)+"_all"
print(model)

loss_fn = torch.nn.BCELoss()

learning_rate = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

####################################################################################################
#Training
####################################################################################################
dev_acc_max=0
epoch_max=0
for e in range(epochs):
	total_loss = 0
	#for x, y in zip(train_x, train_y):
	for i in range(0, train_x.shape[0], N):
		if i+N >= train_x.shape[0]:
			x = train_x[i:]
			y = train_y[i:]
		else:
			x = train_x[i:i+N]
			y = train_y[i:i+N]
		y_pred = model(x.to(device))
		loss = loss_fn(y_pred, y.view(-1,1).to(device))
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
		total_loss += loss.item()
	print("Epoch {}/{}, Train Loss: {:.3f}".format(e+1, epochs, total_loss))
torch.save(model, model_path)
####################################################################################################
#Evaluation
####################################################################################################
with torch.no_grad():
  pdb.set_trace()
  model = torch.load(model_path)
  model.eval().to(device)
  output = model(test_x.to(device))
  output = (output>0.5).float()
  acc    = accuracy_score(test_y, output.cpu())
  print("Test Accuracy: {:.3f}".format(acc))
  print(model)

exit()

In [ ]:
df_train

In [ ]:
df_data.to_csv(path_or_buf="/home/togzhan/sample/data/data/df_data.csv", index=False) 

df_train.to_csv(path_or_buf="/home/togzhan/sample/data/data/df_train3.csv", index=False) 

df_eval.to_csv(path_or_buf="/home/togzhan/sample/data/data/df_eval3.csv", index=False) 


In [41]:
# Adaboost
mx = 5
ns = 200 # 100 - 94% accuracy
lr = 0.6
classifier = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = mx),
    n_estimators=ns,
    learning_rate = lr
)
classifier.fit(train_x, train_y)

predictions = classifier.predict(test_x)

confusion_matrix(test_y, predictions)


array([[14,  4],
       [ 0, 17]])

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics
svc=SVC(probability=True, kernel="sigmoid")
# Adaboost
mx = 5
ns = 200 # 100 - 94% accuracy
lr = 0.6
classifier = AdaBoostClassifier(
    base_estimator=svc,
#     DecisionTreeClassifier(max_depth = mx),
    n_estimators=ns,robotics123
    m
    learning_rate = lr
)
classifier.fit(train_x, train_y)

predictions = classifier.predict(test_x)

confusion_matrix(test_y, predictions)

a = metrics.accuracy_score(test_y, predictions)
print("Accuracy:", a)
a = np.string_(a)

/home/togzhan/anaconda3/envs/python2env/lib/python2.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [35]:
classifier.estimator_errors_

TypeError: 'numpy.ndarray' object is not callable

In [20]:
classifier.staged_score

<bound method AdaBoostClassifier.staged_score of AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=100,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=200, random_state=None)>

In [42]:
a = metrics.accuracy_score(test_y, predictions)
print("Accuracy:", a)
a = np.string_(a)

('Accuracy:', 0.8857142857142857)


In [13]:
model_path = "/home/togzhan/sample/data/data/biclass_adaboost"+str(mx)+"_max_depth"+str(ns)+"_n_estimators"+a+"acc"
torch.save(classifier, model_path)


In [ ]:
predictions